<a href="https://colab.research.google.com/github/gtrujilloes/analyses/blob/main/weekly_seasonal_indexes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
WITH company_sizes AS (SELECT * FROM {{ref('intermediate__company_sizes_historical')}})
, company_attributes AS (SELECT * FROM {{ref('companies_current')}})
, spend_daily AS (SELECT * FROM {{ref('spend_per_customer_daily')}})

, monthly_spend AS (

    SELECT DATE_TRUNC(DATE(performed_date), MONTH)      AS date_month
    , spend_daily.company_id
    , CASE WHEN company_attributes.company_market IN ('GB','DK','DE','NL','ES','FR', 'SE')
            THEN company_attributes.company_market
            ELSE 'OTHER' END                            AS company_market
    , company_sizes.segment_l1
    , company_sizes.segment_l2
    , SUM(spend_eur)                                    AS sum_monthly_spend_eur

    FROM spend_daily

    LEFT JOIN company_attributes
        ON spend_daily.company_id = company_attributes.company_id

    LEFT JOIN company_sizes
        ON company_sizes.company_id = spend_daily.company_id
        AND company_sizes.month = DATE_TRUNC(DATE(spend_daily.performed_date), MONTH)

    WHERE performed_date >= '2022-01-01' --Before 2022 covid had still a very high impact on the business

    GROUP BY 1, 2, 3, 4, 5

    HAVING SUM(spend_eur) > 0 -- only spending customers

)

, outliers AS (

SELECT date_month
    , company_market
    , segment_l1
    , segment_l2
    , APPROX_QUANTILES(sum_monthly_spend_eur, 100)[OFFSET(5)]   AS bottom_5_spending_customers
    , APPROX_QUANTILES(sum_monthly_spend_eur, 100)[OFFSET(95)]  AS top_95_spending_customers

FROM monthly_spend

GROUP BY 1, 2, 3, 4

)

, yearly_average AS (

    SELECT EXTRACT(YEAR FROM monthly_spend.date_month)          AS date_year
        , monthly_spend.segment_l1
        , monthly_spend.segment_l2
        , monthly_spend.company_market
        , AVG(sum_monthly_spend_eur)                            AS avg_yearly_spend_eur

    FROM monthly_spend

    LEFT JOIN outliers
        ON monthly_spend.company_market = outliers.company_market
        AND monthly_spend.segment_l1 = outliers.segment_l1
        AND monthly_spend.segment_l2 = outliers.segment_l2
        AND monthly_spend.date_month = outliers.date_month

    WHERE sum_monthly_spend_eur BETWEEN bottom_5_spending_customers AND top_95_spending_customers --removing outliers for the yearly average

    GROUP BY 1, 2, 3, 4
)

, monthly_seasonal_indexes_per_year AS (

SELECT EXTRACT(YEAR FROM monthly_spend.date_month) AS date_year
    , EXTRACT(MONTH FROM monthly_spend.date_month) AS date_month
    , monthly_spend.segment_l1
    , monthly_spend.segment_l2
    , monthly_spend.company_market
    , yearly_average.avg_yearly_spend_eur
    , SAFE_DIVIDE(AVG(monthly_spend.sum_monthly_spend_eur), yearly_average.avg_yearly_spend_eur)    AS monthly_seasonal_index

FROM monthly_spend

LEFT JOIN outliers
    ON monthly_spend.company_market = outliers.company_market
    AND monthly_spend.segment_l1 = outliers.segment_l1
    AND monthly_spend.segment_l2 = outliers.segment_l2
    AND monthly_spend.date_month = outliers.date_month

LEFT JOIN yearly_average
    ON EXTRACT(YEAR FROM monthly_spend.date_month) = yearly_average.date_year
    AND monthly_spend.company_market = yearly_average.company_market
    AND monthly_spend.segment_l1 = yearly_average.segment_l1
    AND monthly_spend.segment_l2 = yearly_average.segment_l2

WHERE sum_monthly_spend_eur BETWEEN bottom_5_spending_customers AND top_95_spending_customers -- EXCLUDING OUTLIERS

GROUP BY 1, 2, 3, 4, 5, 6

)

SELECT  {{dbt_utils.generate_surrogate_key(['date_month',
                                            'segment_l1',
                                            'segment_l2',
                                            'company_market'])}}    AS finance__spend_monthly_seasonal_indexes_pk
    , date_month
    , segment_l1
    , segment_l2
    , company_market
    , AVG(monthly_seasonal_index)                                   AS monthly_spend_seasonal_index -- calculating the average spend seasonal index for the same month (Jan, Feb..) across all years

FROM monthly_seasonal_indexes_per_year

GROUP BY 1, 2, 3, 4, 5